# High Performance Computing - Coursework (Python Implementation)
##### Raihaan Usman - AE3

> The objective of this coursework assignment is to write a parallel numerical code to solve a 2D reaction-diffusion problem, paying particular attention to computational performance. These types of partial differential equations (PDE) appear in many areas of science including predator-prey scenarios, models of biological systems, excitable media and chemical reactions, etc. They can be quite computationally expensive to solve if the model is stiff (i.e. there are phenomena covering a wide range of timescales), so performance optimisation and parallelisation are important.

The brief sure sounds fun - let's get started!

(P.S. - I'm using Python to prototype the code so I know I've implemented it correctly. I'm not a Python expert, so please don't judge me :)

## 1. Problem Definition

In [1]:
# Mandatory imports :)

import numpy as np
import scipy as sp
import multiprocessing as mp
from matplotlib import pyplot as plt

from solver import ReactionDiffusion

The system of PDEs in question are apparently a variant of the Barkley model and are given by-

\begin{aligned}
&\frac{\partial u}{\partial t}-\mu_{1} \nabla^{2} u=f_{1}(u, v) \\\\
&\frac{\partial v}{\partial t}-\mu_{2} \nabla^{2} v=f_{2}(u, v)
\end{aligned}

where $u_1$ and $u_2$ are coefficients of diffusion, $\mu_{1}$ and $\mu_{2}$ are the reaction rates, and $f_{1}$ and $f_{2}$ are the reaction terms-

\begin{aligned}
&f_{1}(u, v)=\epsilon u(1-u)\left(u-\frac{(v+b)}{a}\right) \\
&f_{2}(u, v)=u^{3}-v
\end{aligned}

In [3]:
# Defining the reaction functions
def _f1(u, v, epsilon, a, b): 
    return epsilon * u * (1-u) * (u - (v+b)/a)


def _f2(u, v):
    return u**3 - v

The specific PDE parameters are $\mu_1, \mu_2, a, b, \epsilon$ and the initial conditions are $u_1(x, y)$ and $u_2(x, y)$.

In [4]:
# Defaults for the system
a       = 0.75
b       = 0.06
mu1     = 5.0
mu2     = 0.0
epsilon = 50

To solve this problem, we're asked to implement forward Euler time marching. So additionally, spatial and temporal discretisation are required - therefore $dx$, $dy$ ($h^2 = dx \cdot dy$) and $dt$ need to be defined, along with $N_x$, $N_y$ and $T$ - the number of discrete nodes in x and y, and the total integration time-

In [5]:
# Discretization parameters
dx      = 1
dy      = 1
Nx      = 11
Ny      = 11
dt      = 1e-3
t       = 10

# Initialise the X, Y and T discrete arrays
X = np.arange(0, Nx, dx)
Y = np.arange(0, Ny, dy)
T = np.arange(0, t+dt, dt)

print(f"\n\
X = \n{X} --> Shape = {X.shape}, \n\n\
Y = \n{Y} --> Shape = {Y.shape}, \n\n\
T = \n{T} --> Shape = {T.shape}")


X = 
[ 0  1  2  3  4  5  6  7  8  9 10] --> Shape = (11,), 

Y = 
[ 0  1  2  3  4  5  6  7  8  9 10] --> Shape = (11,), 

T = 
[0.000e+00 1.000e-03 2.000e-03 ... 9.998e+00 9.999e+00 1.000e+01] --> Shape = (10001,)


## 2. Initial Conditions

Initial conditions are simple enough - for $y > L_y/2 \rightarrow u = 1$ and 0 otherwise. For $x < L_x/2 \rightarrow v = a/2$ and 0 otherwise. 

Note that $L_x$ and $L_y$ are the lengths of the domain in the x and y directions - $N_x * dx$ and $N_y * dy$ respectively.

In [ ]:
# Initialising U and V - in Python, we can use the np.where() function to query the x and y arrays
U = np.where(Y > Ny*dy/2, 1, 0)
U = np.tile(U, (Nx, 1))
U = U.T

V = np.where(X < Nx*dx/2, a/2, 0)
V = np.tile(V, (Ny, 1))

# Note that the indicies correspond to the x and y coordinates and so the matrices appear flipped about the x axis
# print(f"\n\
# Initial U = \n{U} \n\n\
# Initial V = \n{V}")

## 3. Central Differencing

The temporal derivatives are computed by a first-order finite difference scheme, and the Laplacians over the cartesian space are computed using a second-order finite difference scheme.


\begin{aligned}
\frac{u_{i, j}^{n+1}-u_{i, j}^{n}}{\Delta t} &=\frac{\mu_{1}}{h^{2}}\left(u_{i+1, j}^{n}+u_{i-1, j}^{n}+u_{i, j+1}^{n}+u_{i, j-1}^{n}-4 u_{i, j}^{n}\right)+f_{1}\left(u_{i, j}^{n}, v_{i, j}^{n}\right) \\\\
\frac{v_{i, j}^{n+1}-v_{i, j}^{n}}{\Delta t} &=\frac{\mu_{2}}{h^{2}}\left(v_{i+1, j}^{n}+v_{i-1, j}^{n}+v_{i, j+1}^{n}+v_{i, j-1}^{n}-4 v_{i, j}^{n}\right)+f_{2}\left(u_{i, j}^{n}, v_{i, j}^{n}\right) .
\end{aligned}

Pre-multiplying by an identity matrix, for instance, returns an unshifted matrix - e.g $U_{i,j}$. If instead 1's are on the superdiagonal - an upper shift matrix - pre-multiplication with $U$ returns $U_{i+1,j}$. Similarly, if 1's are on the subdiagonal - a lower shift matrix - pre-multiplication with $U$ returns $U_{i-1,j}$.

To shift matrices left or right, i.e in the $j$ direction, the shift matrix can be post-multipied instead. These shifts are sufficient to design a shift matrix, A, to implement this central differencing scheme on the interior points of the mesh-

$$
A := \left[\begin{array}{ccccccc}0 & \cdots & \cdots & \cdots & \cdots & 0 \\1 & -2 & 1 & & & \vdots \\0 & \ddots & \ddots & \ddots & & \vdots \\\vdots & & \ddots & \ddots & \ddots & 0 \\ \vdots & & & 1 & -2 & 1 \\0 & \cdots & \cdots & \cdots & \cdots & 0\end{array}\right]
$$

Note the padded rows and columns - in this form, $AU^n$ or $U^nA$ only computes the $x$ and $y$ components of the Laplacian for the interior nodes of the mesh.

In [ ]:
# Matrix A is a shift matrix which transforms U and V to compute the terms in the central finite difference scheme
# Shape is same as U and V - initalised to zero
A = np.zeros((Nx, Ny))


## 4. Neumann Boundary Conditions

The shift matrix $A$ must be modified to include the boundaries. A Neumann boundary condition is applied to every boundary in $U$ and $V$ ($x=0, x=L_x, y=0, y=L_y$).

This means the normal derivatives of $U$ and $V$ are zero at the boundaries - e.g at $x = 0$, $\frac{\partial{U}}{\partial{x}}$ and $\frac{\partial{V}}{\partial{x}}$ are zero. In discrete form, this is given by-

$\begin{aligned}
\left(u_{1, j}^{n}-u_{-1, j}^{n}\right) / h = 0 ; \quad u_{-1, j}^{n} = u_{0, j}^{n}
\end{aligned}$

$\begin{aligned}
\therefore \frac{u_{0, j}^{n+1}-u_{0, j}^{n}}{\Delta t} = \frac{\mu_{1}}{h^{2}}\left(u_{1, j}^{n}+u_{0, j}^{n}+u_{0, j+1}^{n}+u_{0, j-1}^{n}-2 u_{0, j}^{n}\right)+f_{1}\left(u_{0, j}^{n}, v_{i, j}^{n}\right)
\end{aligned}$


The modified shift matrix is adjusted to account for the updated central difference scheme at the boundaries. Note that it is still a symmetric banded matrix-

$$
A := \left[\begin{array}{ccccccc}-1 & 1 & 0 & \cdots & \cdots & 0 \\1 & -2 & 1 & & & \vdots \\0 & \ddots & \ddots & \ddots & & \vdots \\\vdots & & \ddots & \ddots & \ddots & 0 \\ \vdots & & & 1 & -2 & 1 \\\\0 & \cdots & \cdots & 0 & 1 & -1\end{array}\right]
$$

## 5. Time Integration

Finally, forward Euler time marching is implemented using a simple loop over the time steps-

$\begin{aligned}
u_{i,j}^{n+1} = u_{i,j}^{n} + \frac{u_{i, j}^{n+1}-u_{i, j}^{n}}{\Delta t} \cdot dt
\end{aligned}$

In [ ]:
# Time marching with forward Euler
def forward_euler(U, V, f1, f2, dt, T):
    '''
    Solve the PDE system using forward Euler
    '''
    
    # Initialise the solution arrays
    U_sol = np.zeros((Nx, Ny))
    V_sol = np.zeros((Nx, Ny))

    # Initialise the time array
    t = 0

    # Loop through the time array
    for i in range(len(T)):
        # Update the solution arrays
        U_sol[:, :] = U
        V_sol[:, :] = V

        # Update the solution arrays
        U += dt*f1(U, V, epsilon, a, b)
        V += dt*f2(U, V)

        # Print the time and the solution arrays
        print(f"\n\
        t = {t} \n\
        U = \n{U} \n\
        V = \n{V}")

        # Increment the time
        t += dt

    return U_sol, V_sol